In [ ]:
from pathlib import Path

import cf_xarray.units
import xarray as xr
import pint_xarray
import numpy as np

from pism_tutorials.utils import merge_dicts, dict2str, sort_dict_by_key

In [ ]:
rho_i = xr.DataArray(918.).pint.quantify("kg m^-3")
rho_sw = xr.DataArray(1028.).pint.quantify("kg m^-3")
accum = xr.DataArray(0.3).pint.quantify("m yr^-1") 
cmb = (accum * rho_i)
ice_surface_temp = xr.DataArray(268.13).pint.quantify("K")
A = xr.DataArray(6.338e-25).pint.quantify("Pa^−3 s^−1")
to = xr.DataArray(277.15).pint.quantify("K")
so = xr.DataArray(35).pint.quantify("g/kg")
b = xr.DataArray(1).pint.quantify("")


def Bx(x, B0 = -150, B2 = -728.8, B4 = 343.91, B6 = -50.57, x_bar = 300e3):
    
    return B0 + B2 * (x / x_bar)**2  + B4 * (x / x_bar)**4 + B6 * (x / x_bar)**6

def By(y, Ly = 80e3, dc = 500, fc = 4e3, wc = 24e3):
    return dc / (1 + np.exp(-2*(y -Ly/2 - wc) /fc)) + dc / (1 + np.exp(2*(y -Ly/2 + wc) /fc))


Lx = 640e3
Ly = 80e3
dx = 2e3
dy = 2e3
x = np.arange(0, Lx + 160e3, dx)
y = np.arange(0, Ly, dy)

X, Y = np.meshgrid(x, y)

bed_deep = -720


bed = np.maximum(Bx(X) + By(Y), bed_deep)
liafr = np.zeros_like(bed)
liafr[X<=Lx] = 1
thickness = np.zeros_like(bed) + 100

In [ ]:
x_dim = "x"
y_dim = "y"

coords = {
    x_dim: (
        x_dim,
        x,
        {
            "units": "m",
            "axis": x_dim.upper(),
            "standard_name": "projection_x_coordinate",
            "long_name": f"{x_dim}-coordinate in projected coordinate system",
        },
    ),
    y_dim: (
        y_dim,
        y,
        {
            "units": "m",
            "axis": y_dim.upper(),
            "standard_name": "projection_y_coordinate",
            "long_name": f"{y_dim}-coordinate in projected coordinate system",
        },
    ),
}

ds = xr.Dataset(
    {
        "bed": xr.DataArray(
            data=bed,
            dims=[y_dim, x_dim],
            coords={y_dim: coords[y_dim], x_dim: coords[x_dim]},
            attrs={
                "standard_name": "bedrock_altitude",
                "units": "m"
            },
        ),
        "thickness": xr.DataArray(
            data=thickness,
            dims=[y_dim, x_dim],
            coords={y_dim: coords[y_dim], x_dim: coords[x_dim]},
            attrs={
                "standard_name": "land_ice_thickness",
                "units": "m"
            },
        ),
        "land_ice_area_fraction_retreat": xr.DataArray(
            data=liafr,
            dims=[y_dim, x_dim],
            coords={y_dim: coords[y_dim], x_dim: coords[x_dim]},
            attrs={
                "units": "1",
            },
        ),            
    },
    attrs={"Conventions": "CF-1.8"}
)
ds["climatic_mass_balance"] = xr.zeros_like(ds["bed"]) + cmb.broadcast_like(ds["bed"])
ds["ice_surface_temp"] = xr.zeros_like(ds["bed"]) + ice_surface_temp.broadcast_like(ds["bed"])
ds = ds.pint.dequantify()
ds["climatic_mass_balance"].attrs.update({"units": "kg m^-2 yr^-1"})
ds.to_netcdf("mismip+.nc")

theta_ocean = xr.zeros_like(ds["bed"]) + to
theta_ocean.name = "theta_ocean"
theta_ocean.attrs.update({"units": "K"})
salinity_ocean =  xr.zeros_like(ds["bed"]) + so
salinity_ocean.name = "salinity_ocean"
salinity_ocean.attrs.update({"units": "g/kg"})

basins =  xr.zeros_like(ds["bed"]) + b
basins.name = "basins"

ocean_ds = xr.merge([theta_ocean, salinity_ocean, basins])
ocean_ds.to_netcdf("pico.nc")

In [ ]:
resolution = "2km"
start = 0
end = 5000
output_dir = Path("mismip+")
output_dir.mkdir(parents=True, exist_ok=True)
alpha = 0.5


extra_file = output_dir / Path(f"spatial_g{resolution}_{start}_{end}.nc")
state_file = output_dir / Path(f"state_g{resolution}_{start}_{end}.nc")
                               
input_params = {
    "bootstrap": "",
    "i": "mismip+.nc",
    "geometry.front_retreat.prescribed.file": "mismip+.nc",
    "geometry.front_retreat.use_cfl": "",
    "geometry.part_grid.enabled": "",
    "geometry.remove_icebergs": "",
    "grid.registration": "corner",
}

grid_params = {
    "grid.dx": resolution,
    "grid.dy": resolution,
    "grid.Mz": 601,
    "grid.Lz": 6000,
    "grid.Mbz": 11,
    "grid.Lbz": 1000,
}

time_params = {
    "time.start": start,
    "time.end": end,
    "time_stepping.skip.enabled": "",
    "time_stepping.skip.max": 100
}

climate_params = {
    "atmosphere.models": "given",
    "surface.models": "given",
    "surface.given.file": "mismip+.nc",
    "ocean.models": "constant",
    "ocean.constant.melt_rate": 0.0,
}

dynamics_params = {
    "constants.ice.density": rho_i.pint.dequantify().values,
    "constants.sea_water.density": rho_sw.pint.dequantify().values,
    "basal_resistance.pseudo_plastic.enabled": "yes",
    "basal_resistance.regularized_coulomb.enabled": "yes",
    "basal_resistance.pseudo_plastic.q": 1./3.,
    "basal_resistance.pseudo_plastic.u_threshold": "100m/yr",
    "basal_yield_stress.constant.value": 1e7,
    "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": alpha**2,
    "energy.model": "none",
    "flow_law.isothermal_Glen.ice_softness": A.pint.dequantify().values,
    "stress_balance.model": "ssa",
    "stress_balance.sia.max_diffusivity": 1e5,
    "stress_balance.sia.surface_gradient_method": "eta",
    "stress_balance.sia.flow_law": "isothermal_glen",
    "stress_balance.ssa.flow_law": "isothermal_glen",
    "stress_balance.blatter.flow_law": "isothermal_glen",
    "stress_balance.blatter.coarsening_factor": 4,
    "stress_balance.blatter.Mz": 17,
    "bp_ksp_type": "gmres",
    "bp_pc_type": "mg",
    "bp_pc_mg_levels": 3,
    "bp_mg_levels_ksp_type": "richardson",
    "bp_mg_levels_pc_type": "sor", 
    "bp_mg_coarse_ksp_type": "preonly",
    "bp_mg_coarse_pc_type": "lu", 
    "bp_snes_monitor_ratio": "",
    "bp_ksp_monitor": "",
    "bp_ksp_view_singularvalues": "",
    "bp_snes_ksp_ew": 1,
    "bp_snes_ksp_ew_version": 3,
    "time_stepping.adaptive_ratio": 250,
    "stress_balance.calving_front_stress_bc": "",  
}

hydrology_params = {"hydrology.model": "null",
                   }

output_params = {
    "output.extra.file": extra_file,
    "output.extra.times": "100year",
    "output.extra.vars": "bmelt,mask,topg,usurf,thk,velsurf_mag,velbase_mag,climatic_mass_balance,taub_mag",
    "output.sizes.medium": "sftgif,velsurf_mag,mask,usurf,bmelt",
    "output.file": state_file
}

run_dict = merge_dicts(input_params, grid_params, time_params, dynamics_params, hydrology_params, climate_params, output_params)

In [ ]:
n = 8
run_str = dict2str(sort_dict_by_key(run_dict))
cmd = f"mpirun -np {n} pism " + run_str
! $cmd

In [ ]:
resolution = "1km"
start = end
end = start + 100

infile = state_file
extra_file = output_dir / Path(f"spatial_g{resolution}_{start}_{end}.nc")
state_file = output_dir / Path(f"state_g{resolution}_{start}_{end}.nc")

run_dict.update({    
    "time.start": start,
    "time.end": end,
    "grid.dx": resolution,
    "grid.dy": resolution,
    "output.extra.file": extra_file,
    "output.extra.times": "10year",
    "output.file": state_file,
    "i": infile}
)

In [ ]:
run_str = dict2str(sort_dict_by_key(run_dict))
cmd = f"mpirun -np {n} pism " + run_str
! $cmd

In [ ]:
resolution = "1km"
start = end
end = start + 100

infile = state_file
output_dir = Path("mismip+_pico")
output_dir.mkdir(parents=True, exist_ok=True)

extra_file = output_dir / Path(f"spatial_g{resolution}_{start}_{end}.nc")
state_file = output_dir / Path(f"state_g{resolution}_{start}_{end}.nc")

run_dict.update({    
    "ocean.models": "pico",
    "ocean.pico.file": "pico.nc",
    "ocean.pico.heat_exchange_coefficent": 1.63e-3,
    "ocean.pico.number_of_boxes": 5,
    "ocean.pico.overturning_coefficent": 1e6,
    "time.start": start,
    "time.end": end,
    "grid.dx": resolution,
    "grid.dy": resolution,
    "output.extra.file": extra_file,
    "output.extra.times": "1year",
    "output.file": state_file,
    "i": infile}
)

In [ ]:
run_str = dict2str(sort_dict_by_key(run_dict))
cmd = f"mpirun -np {n} pism " + run_str
! $cmd

In [ ]:
!$cmd